***Challenge 1***

Here the goal is to train on 25 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 25 from a datasource that is not the CIFAR-10 training data.

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset.

In [ ]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CrossAttentionModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=256, num_heads=8)

    def forward(self, query, key, value):
        return self.attention(query, key, value)[0]

class CAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*32*3, 256),
            nn.ReLU(),
        )
        self.cam = CrossAttentionModule()
        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        features = self.feature_extractor(x)
        attention_output = self.cam(features.unsqueeze(1), features.unsqueeze(1), features.unsqueeze(1))
        out = self.classifier(attention_output.squeeze(1))
        return out

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
import time


from torchvision import datasets, transforms
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)

#We need two copies of this due to weird dataset api
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)


accs = []

times = []

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128,
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128,
                                           shuffle=False)


  model = CAN().to(device)
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  start_time = time.time()
  for epoch in range(100):
    train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)

  end_time = time.time()

  times.append(end_time - start_time)

  accs.append(test(model, device, val_loader))

times = np.array(times)
accs = np.array(accs)
print('Acc over 5 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +-{times.std()}")


100%|██████████| 170498071/170498071 [00:03<00:00, 44011351.98it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.299343
Train Epoch: 5 [0/50 (0%)]	Loss: 1.485493
Train Epoch: 10 [0/50 (0%)]	Loss: 0.318496
Train Epoch: 15 [0/50 (0%)]	Loss: 0.094607
Train Epoch: 20 [0/50 (0%)]	Loss: 0.031581
Train Epoch: 25 [0/50 (0%)]	Loss: 0.003033
Train Epoch: 30 [0/50 (0%)]	Loss: 0.000472
Train Epoch: 35 [0/50 (0%)]	Loss: 0.000159
Train Epoch: 40 [0/50 (0%)]	Loss: 0.000050
Train Epoch: 45 [0/50 (0%)]	Loss: 0.000011
Train Epoch: 50 [0/50 (0%)]	Loss: 0.000004
Train Epoch: 55 [0/50 (0%)]	Loss: 0.000003
Train Epoch: 60 [0/50 (0%)]	Loss: 0.000002
Train Epoch: 65 [0/50 (0%)]	Loss: 0.000002
Train Epoch: 70 [0/50 (0%)]	Loss: 0.000002
Train Epoch: 75 [0/50 (0%)]	Loss: 0.000002
Train Epoch: 80 [0/50 (0%)]	Loss: 0.000001
Train Epoch: 85 [0/50 (0%)]	Loss: 0.000001
Train Epoch: 90 [0/50 (0%)]	Loss: 0.000001
Train Epoch: 95 [0/50 (0%)]	Loss: 0.000001


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 3.6626, Accuracy: 248/400 (62.00%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.316191
Train Epoch: 5 [0/50 (0%)]	Loss: 1.119318
Train Epoch: 10 [0/50 (0%)]	Loss: 0.285347
Train Epoch: 15 [0/50 (0%)]	Loss: 0.024102
Train Epoch: 20 [0/50 (0%)]	Loss: 0.010567
Train Epoch: 25 [0/50 (0%)]	Loss: 0.002549
Train Epoch: 30 [0/50 (0%)]	Loss: 0.000236
Train Epoch: 35 [0/50 (0%)]	Loss: 0.000084
Train Epoch: 40 [0/50 (0%)]	Loss: 0.000049
Train Epoch: 45 [0/50 (0%)]	Loss: 0.000021
Train Epoch: 50 [0/50 (0%)]	Loss: 0.000014
Train Epoch: 55 [0/50 (0%)]	Loss: 0.000012
Train Epoch: 60 [0/50 (0%)]	Loss: 0.000010
Train Epoch: 65 [0/50 (0%)]	Loss: 0.000009
Train Epoch: 70 [0/50 (0%)]	Loss: 0.000009
Train Epoch: 75 [0/50 (0%)]	Loss: 0.000008
Train Epoch: 80 [0/50 (0%)]	Loss: 0.000007
Train Epoch: 85 [0/50 (0%)]	Loss: 0.000006
Train Epoch: 90 [0/50 (0%)]	Loss: 0.000005
Train Epoch: 95 [0/50 (0%)]	Loss: 0.000005

Test set: Average loss: 3.23